# Requirements and Setup

In order to execute this notebook, make sure you have already installed the necessary requirements described in the README.md.
The following are all the necessary imports to run the entire notebook, from start to end.

In [ ]:
import os 
import glob
import pandas as pd 
import numpy as np
from main.tree.linkageTree import linkageCut
from main.tsp.TSP_Formulation_Methods import ( 
    create_QUBO_matrix,
    solve_qubo_with_Dwave,
    check_solution,
    check_solution_return,
    load_lambda_means,
    draw_solution_graph,
    brute_force_finding,
    calculate_distances_cost
)
from main.vqaa.vqaa_tools import ( 
    heuristical_embedding, 
    atoms_register,
    atoms_list, 
    generate_grid,
    run_vqaa,
    plot_distribution,
)
from main.tree.utils import ( 
    view_linkage_on_map, 
    draw_centers_on_map,
    map_draw_line,
    convert_bitstring_to_matrix,
    assemble_line,
    string_to_bitstring,
)
from data.utils import fetch_amenities_from

# Load initial data 

This notebooks shows the proof-of-concept of the bus line optimization algorithm developed. To start, we must first fetch data from a specific city.
In this case, we fetch the data from Granada, Spain. The data consists of local amenities.

In [ ]:
# Load previously stored overpy lat/lon datafile for different amenities
filepath = os.path.join('data', 'amenities-granada.csv')
if os.path.exists(filepath):
    amenities_data = pd.read_csv(filepath)
else:
    # If there is no previous data:
    query_file = os.path.join('data', 'overpy-granada-query.txt')
    query = None
    with open(query_file) as file:
        query = file.read()
    amenities_data = fetch_amenities_from(query=query) # Defaults to Granada
    amenities_data.to_csv(filepath)
# Create a hierarchical clustering of amenities
hierarchical_cluster = linkageCut(amenities_data)
# Set a specific number of clusters per levels. Max 9 in this POC
nclusters = 4
levels = 2
labels = hierarchical_cluster.top_down_view_recur(nclusters=nclusters, levels=2)
# Visualize for debugging purposes.
view_linkage_on_map(linkage_matrix=hierarchical_cluster)

# Level 0 solutions

In [ ]:
# Fetch the centers of the first level
centers = hierarchical_cluster.give_centers_level(0)
# Sanity check by drawing the graph
draw_centers_on_map(centers, color='red')

For our example we will impose two random nodes as start and endpoints.
Say we wanted to connect a marginalized area with another area where public services or/and green spaces are available. These extreme nodes will work as the extremes of the route. Additionally, we will fix for the route to traverse a fixed (p) number of districts (level-0 clusters).

In [ ]:
np.random.seed(54)
# Fetch the distance from the centers of the first level
distances = hierarchical_cluster.dist_matrix_level(0, return_labels=False)
# Set initial global parameters
N = distances.shape[0]
p = 2
node_options = set(np.arange(nclusters) + 1)
startNode = np.random.choice(list(node_options))
endNode = np.random.choice(list(node_options - set([startNode])))
print("Selected random nodes:", startNode, endNode)

# Process Parameters
p = min(p, N-1)
startNode = min(startNode, N)
endNode = min(endNode, N)

reduced_distances = distances/np.max(distances)
maxDistance = np.max(reduced_distances)
# NOTE: temporary double pardir while we decide the new structure for the lambdas
lambda_paths = glob.glob(os.path.join(os.path.pardir, 'data', 'lamdasOptimized', '*'))
mean_lambdas = load_lambda_means(lambda_paths)

The general procedure is shown here. First, the QUBO matrix is generated, with constraints penalties optimized for the problem (see docs).
Second, we embed the atoms and then we use any of the available emulators to solve the QUBO problem. In this case, we compare the solution with the brute force result. The brute fore solution will only be computed when $N\cdot p < 15$, since we have to check $2^{N\cdot p}$ combinations. Then, we can plot the raw result: the number of counts.

WARNING: The simulator may explode sometimes giving 0000.. as only counts

In [ ]:
Q_matrix_initial,_ = create_QUBO_matrix(reduced_distances, p, startNode - 1, endNode - 1, mean_lambdas)   
if N*(p+1) < 20:
    # Getting first solution, the one with lower cost
    solution_ref = brute_force_finding(Q_matrix_initial, reduced_distances, p, False)[0][0]
    
else:
    solution_ref, _ = solve_qubo_with_Dwave(Q_matrix_initial, num_reads=1000)
print('Our solution reference is:')
print(solution_ref)
check_solution(string_to_bitstring(solution_ref), N,  p, startNode - 1, endNode - 1)

# Creating register and solving VQAA
coords = heuristical_embedding(atoms_list(len(Q_matrix_initial)), generate_grid(50, 50,1), Q_matrix_initial)
register = atoms_register(coords)
# Emulator options: "qutip" (pulser), "mps", and "sv"
C_0, x = run_vqaa(Q_matrix_initial, register, "qutip")

plot_distribution(C_0, 50, "".join([str(x) for x in solution_ref]))

Now that we have the set of proposed solutions, the ones with higher counts (in this case, 
we will take the top 100) we can  verify our imposed QUBO constraints. Furthermore, from our surviving options, we can select the 
one with lower associated cost in terms of distance: our main objective! We explicitely define the function to highlight its simplicity.

In [ ]:
def get_best_solution(proposed_sols, distances, p):
    # Discard solutions that do not fulfill our constraints
    constrained = [check_solution_return(string_to_bitstring(sol), N, p, startNode - 1, endNode - 1) for sol in proposed_sols]
    # Select the one(s) with lower distance associated cost
    distance_costs = np.array([calculate_distances_cost(np.array(string_to_bitstring(sol)), distances, p) for sol in proposed_sols])
    print(distance_costs[constrained])
    if np.sum(constrained) == 0:
        print('NO solution, consider increasing the number of proposed solutions')
    return proposed_sols[constrained][np.argmin(distance_costs[constrained])]

number_props = len(C_0.keys())
print(number_props)
C_ = dict(sorted(C_0.items(), key=lambda item: item[1], reverse=True))
proposed_sols = np.array(list(C_.keys() ) )[:number_props]
level0_sols = get_best_solution(proposed_sols, distances, p) # symmetric matrix means that we are counting distances twice
level0_sols = string_to_bitstring(level0_sols)
print("Found solution:", level0_sols)

In [ ]:
adjacency = convert_bitstring_to_matrix(level0_sols, N=N, p=p)
map_draw_line(centers[:, ::-1], adjacency, color='red')

# Level 1 solutions

In [ ]:
# Formulation with initial lambdas
level1_sols = {} # Dict that will hold the bitstring, connected level-0 clusters and corresponding start-end nodes 
all_indices = set(np.arange(nclusters - 1) + 1)
for i in range(1, nclusters+1):
    
    connections = np.concatenate([adjacency[:,i-1].nonzero()[0], adjacency[i-1, :].nonzero()[0]], axis=0) + 1
    print("----- Solving level-1:", i, "------\n")
    print("connections", connections)
    if len(connections) > 0: #Selected
        # Fetch the centers of the first level
        distances, closest, _ = hierarchical_cluster.dist_matrix_label_down(
        i,
        connections=connections,
        )
        
        startNode = None
        if len(closest) >= 1:
            startNode = closest[0]
            choices = all_indices - set([startNode])
        if len(closest) == 2:
            endNode = closest[1]
        else:
            endNode = np.random.choice(list(choices)) # POC criterion, better heuristic should be chosen
        print("Start-end", startNode, endNode)

        Q_matrix_initial,_ = create_QUBO_matrix(reduced_distances, p, startNode - 1, endNode - 1, mean_lambdas)
        
        # Creating register and solving VQAA
        coords = heuristical_embedding(atoms_list(len(Q_matrix_initial)), generate_grid(50, 50,1), Q_matrix_initial)
        register = atoms_register(coords)
        C_1, x = run_vqaa(Q_matrix_initial, register, "qutip")
        
        # Our ref is Dwave
        solution_ref = solve_qubo_with_Dwave(Q_matrix_initial, num_reads=1000)
        plot_distribution(C_1, 50, "".join([str(x) for x in solution_ref]))
        
        C_ = dict(sorted(C_1.items(), key=lambda item: item[1], reverse=True))
        number_props = len(C_.keys())
        proposed_sols = np.array(list(C_.keys() ) )[:number_props]
        sol_ =  get_best_solution(proposed_sols, distances, p) # symmetric matrix means that we are counting distances twice
        print("Proposed solution bitstring:")
        print(sol_)
        sol_ = string_to_bitstring(sol_)
        level1_sols[i] = [sol_, closest]
        
    else:
        level1_sols[i] = (np.zeros((nclusters*(p + 1))), [])
        print('The line does not cross this level-0 cluster')


This is how one route at level 1 looks like. Feel free to try different example_indx. The clusters where the line is defined are:

In [ ]:
print(np.nonzero(np.array(level0_sols).reshape(p+1, N))[1] + 1)

In [ ]:
example_indx = 2
example_centers = hierarchical_cluster.give_centers_label_down(example_indx)
example_adj = convert_bitstring_to_matrix(level1_sols[example_indx][0],N=nclusters, p=p)
map = map_draw_line(centers[:, ::-1], adjacency, color='red')
map_draw_line(example_centers[:, ::-1], example_adj, color='blue', map=map)

Let's piece all the information about the calculated lines at level 1 together using what we know about the connections at level 0. Effectively creating a unified route.

How do we join all these disconnected level-1 mini-routes? The larger-scale route is given by the level 0. We know that 1 should be connected with 2, then we can connect the closest (driving distance) clusters.

In [ ]:
assembled = assemble_line(np.array(level0_sols),level1_sols, nclusters, p)

In [ ]:
centers_level1 = hierarchical_cluster.give_centers_level(1)
map_draw_line(centers_level1[:,::-1], assembled, color='blue')